In [4]:
import pulp

# 创建问题
model = pulp.LpProblem(name="nurse-scheduling-plan3", sense=pulp.LpMinimize)

# 决策变量
# 方案二类型的护士：x[i,j]表示第i种休息组合，从第j个班次开始上班的护士数量
# i=1-5: 周六休息，周一到周五某天休息
# i=6-10: 周日休息，周一到周五某天休息
# j=1-5: 从第j个班次开始上班
x = {}
for i in range(1, 11):
    for j in range(1, 6):
        x[i, j] = pulp.LpVariable(f"x_{i}_{j}", lowBound=0, cat=pulp.LpInteger)

# 方案三类型的护士：y[i,j]表示周一至周五中休息两天的组合i，从第j个班次开始上班的护士数量
# i=1-10: 代表从周一至周五中选择两天休息的10种组合
# j=1-5: 从第j个班次开始上班
y = {}
for i in range(1, 11):
    for j in range(1, 6):
        y[i, j] = pulp.LpVariable(f"y_{i}_{j}", lowBound=0, cat=pulp.LpInteger)

# 目标函数：最小化所需护士总数
model += pulp.lpSum(x[i, j] for i in range(1, 11) for j in range(1, 6)) + pulp.lpSum(y[i, j] for i in range(1, 11) for j in range(1, 6))

# 生成周一至周五中选择两天休息的所有组合
def generate_rest_combinations():
    combinations = []
    for i in range(1, 6):
        for j in range(i+1, 6):
            combinations.append((i, j))
    return combinations

rest_combinations = generate_rest_combinations()

# 辅助函数：判断护士x是否在指定日期和时间段工作
def is_x_on_duty(rest_combo, start_shift, day, time_slot):
    # 确定护士的休息日
    if rest_combo <= 5:  # 周六休息 + 周一到周五某天休息
        saturday_rest = True
        weekday_rest = rest_combo  # 1-5 对应周一到周五
    else:  # 周日休息 + 周一到周五某天休息
        saturday_rest = False
        weekday_rest = rest_combo - 5  # 6-10 对应周一到周五
    
    # 确定护士的工作日
    work_days = []
    for d in range(1, 8):  # 1-7 对应周一到周日
        if d == 6 and saturday_rest:  # 周六休息
            continue
        if d == 7 and not saturday_rest:  # 周日休息
            continue
        if d == weekday_rest:  # 周一到周五某天休息
            continue
        work_days.append(d)
    
    # 确定护士的工作班次
    shifts = []
    for s in range(5):  # 5个班次
        shift_day = (day - 1 + s) % 7 + 1  # 计算班次对应的日期
        if shift_day in work_days:
            shifts.append((s + 1) % 5 + 1)  # 1-5 对应5个班次
    
    # 判断护士是否在指定日期和时间段工作
    if day in work_days:
        shift_index = (day - 1 + start_shift - 1) % 5
        shift = shifts[shift_index] if shift_index < len(shifts) else None
        
        if shift is not None:
            # 判断班次是否覆盖指定时间段
            if time_slot == "6:00-10:00" and shift in [1, 2]:
                return True
            elif time_slot == "10:00-14:00" and shift in [2, 3]:
                return True
            elif time_slot == "14:00-18:00" and shift in [3, 4]:
                return True
            elif time_slot == "18:00-22:00" and shift in [4, 5]:
                return True
            elif time_slot == "22:00-6:00" and shift in [5, 1]:
                return True
    
    # 特殊处理夜班跨越午夜的情况
    if time_slot == "22:00-6:00":
        prev_day = day - 1 if day > 1 else 7  # 前一天
        
        # 如果前一天是工作日
        if prev_day in work_days:
            prev_shift_index = (prev_day - 1 + start_shift - 1) % 5
            prev_shift = shifts[prev_shift_index] if prev_shift_index < len(shifts) else None
            
            # 如果前一天是第5班（18:00-2:00）
            if prev_shift == 5:
                return True
    
    return False

# 辅助函数：判断护士y是否在指定日期和时间段工作
def is_y_on_duty(rest_combo_index, start_shift, day, time_slot):
    # 获取休息日组合
    rest_combo = rest_combinations[rest_combo_index - 1]
    weekday_rest1, weekday_rest2 = rest_combo
    
    # 确定护士的工作日
    work_days = []
    for d in range(1, 8):  # 1-7 对应周一到周日
        if d == weekday_rest1 or d == weekday_rest2:  # 周一到周五中休息的两天
            continue
        work_days.append(d)
    
    # 确定护士的工作班次
    shifts = []
    for s in range(5):  # 5个班次
        shift_day = (day - 1 + s) % 7 + 1  # 计算班次对应的日期
        if shift_day in work_days:
            shifts.append((s + 1) % 5 + 1)  # 1-5 对应5个班次
    
    # 判断护士是否在指定日期和时间段工作
    if day in work_days:
        shift_index = (day - 1 + start_shift - 1) % 5
        shift = shifts[shift_index] if shift_index < len(shifts) else None
        
        if shift is not None:
            # 判断班次是否覆盖指定时间段
            if time_slot == "6:00-10:00" and shift in [1, 2]:
                return True
            elif time_slot == "10:00-14:00" and shift in [2, 3]:
                return True
            elif time_slot == "14:00-18:00" and shift in [3, 4]:
                return True
            elif time_slot == "18:00-22:00" and shift in [4, 5]:
                return True
            elif time_slot == "22:00-6:00" and shift in [5, 1]:
                return True
    
    # 特殊处理夜班跨越午夜的情况
    if time_slot == "22:00-6:00":
        prev_day = day - 1 if day > 1 else 7  # 前一天
        
        # 如果前一天是工作日
        if prev_day in work_days:
            prev_shift_index = (prev_day - 1 + start_shift - 1) % 5
            prev_shift = shifts[prev_shift_index] if prev_shift_index < len(shifts) else None
            
            # 如果前一天是第5班（18:00-2:00）
            if prev_shift == 5:
                return True
    
    return False

# 辅助函数：获取某天某时间段工作的护士数量
def get_nurses_on_duty(day, time_slot):
    nurses = 0
    
    # 遍历所有x类型的护士
    for i in range(1, 11):
        for j in range(1, 6):
            if is_x_on_duty(i, j, day, time_slot):
                nurses += x[i, j]
    
    # 遍历所有y类型的护士
    for i in range(1, 11):
        for j in range(1, 6):
            if is_y_on_duty(i, j, day, time_slot):
                nurses += y[i, j]
    
    return nurses

# 添加约束条件
time_slots = ["6:00-10:00", "10:00-14:00", "14:00-18:00", "18:00-22:00", "22:00-6:00"]
min_nurses = {"6:00-10:00": 18, "10:00-14:00": 20, "14:00-18:00": 19, "18:00-22:00": 17, "22:00-6:00": 12}

for day in range(1, 8):  # 1-7 对应周一到周日
    for time_slot in time_slots:
        model += get_nurses_on_duty(day, time_slot) >= min_nurses[time_slot]

# 求解问题
model.solve()

# 输出结果
print("方案3求解状态:", pulp.LpStatus[model.status])

# 统计方案二类型的护士数量
type_x_nurses = 0
for i in range(1, 11):
    for j in range(1, 6):
        if pulp.value(x[i, j]) > 0:
            if i <= 5:
                print(f"x_{i}_{j} (周六休息，周{i}休息，从第{j}班开始): {pulp.value(x[i, j])}")
            else:
                print(f"x_{i}_{j} (周日休息，周{i-5}休息，从第{j}班开始): {pulp.value(x[i, j])}")
            type_x_nurses += pulp.value(x[i, j])

# 统计方案三类型的护士数量
type_y_nurses = 0
for i in range(1, 11):
    for j in range(1, 6):
        if pulp.value(y[i, j]) > 0:
            rest_days = rest_combinations[i-1]
            print(f"y_{i}_{j} (周一至周五中休息第{rest_days[0]}天和第{rest_days[1]}天，从第{j}班开始): {pulp.value(y[i, j])}")
            type_y_nurses += pulp.value(y[i, j])

print(f"方案3需要的方案二类型护士数量: {type_x_nurses}")
print(f"方案3需要的方案三类型护士数量: {type_y_nurses}")
print(f"方案3需要的护士总数: {type_x_nurses + type_y_nurses}")

# 计算a值
# 假设护士的基本工资为w
# 方案二的总工资成本：w * type_x_nurses
# 方案三的总工资成本：w * type_x_nurses + w * (1 + a/100) * type_y_nurses
# 令两者相等，求解a
if type_y_nurses > 0:

    plan2_total_nurses = 105 
    
    # 计算a值
    if plan2_total_nurses > type_x_nurses + type_y_nurses:
        a = ((plan2_total_nurses - type_x_nurses - type_y_nurses) / type_y_nurses) * 100
        print(f"当a≥{a:.2f}%时，方案3比方案2更经济")
    else:
        print("方案3所需护士人数比方案2少，即使a=0，方案3也更经济")

方案3求解状态: Optimal
x_1_1 (周六休息，周1休息，从第1班开始): 6.0
x_1_3 (周六休息，周1休息，从第3班开始): 5.0
x_1_5 (周六休息，周1休息，从第5班开始): 4.0
x_2_3 (周六休息，周2休息，从第3班开始): 2.0
x_4_3 (周六休息，周4休息，从第3班开始): 1.0
x_4_5 (周六休息，周4休息，从第5班开始): 2.0
x_6_5 (周日休息，周1休息，从第5班开始): 1.0
x_7_2 (周日休息，周2休息，从第2班开始): 4.0
x_7_4 (周日休息，周2休息，从第4班开始): 5.0
x_8_3 (周日休息，周3休息，从第3班开始): 2.0
x_9_2 (周日休息，周4休息，从第2班开始): 3.0
x_9_3 (周日休息，周4休息，从第3班开始): 2.0
x_10_1 (周日休息，周5休息，从第1班开始): 1.0
x_10_2 (周日休息，周5休息，从第2班开始): 3.0
x_10_4 (周日休息，周5休息，从第4班开始): 4.0
y_1_1 (周一至周五中休息第1天和第2天，从第1班开始): 4.0
y_1_4 (周一至周五中休息第1天和第2天，从第4班开始): 4.0
y_1_5 (周一至周五中休息第1天和第2天，从第5班开始): 2.0
y_2_5 (周一至周五中休息第1天和第3天，从第5班开始): 3.0
y_5_3 (周一至周五中休息第2天和第3天，从第3班开始): 3.0
y_6_2 (周一至周五中休息第2天和第4天，从第2班开始): 2.0
y_8_2 (周一至周五中休息第3天和第4天，从第2班开始): 4.0
y_9_1 (周一至周五中休息第3天和第5天，从第1班开始): 6.0
y_10_1 (周一至周五中休息第4天和第5天，从第1班开始): 3.0
方案3需要的方案二类型护士数量: 45.0
方案3需要的方案三类型护士数量: 31.0
方案3需要的护士总数: 76.0
当a≥93.55%时，方案3比方案2更经济
